In [1]:
import numpy as np  # to create a matrix with zero elements
import random
import pygame # implement shape or graphics of game
import sys
import math

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# colors we need in the game
BLUE = (0 , 0 , 255)  # to create a blue rectangle
BLACK = (0 , 0 , 0)   # to create a black circles of the game
RED = (225 , 0 , 0)   # to create a red circle for player 1
YELLOW = (225 , 225 , 0) # to craete a yellow circle for player 2

In [ ]:
# no of rows and columns of the game
ROW_COUNT = 6
COLUMN_COUNT = 7

In [ ]:
#AI for turn
PLAYER = 0
AI = 1

In [ ]:
#AI in matrix
EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

In [ ]:
#AI
WINDOW_LENGTH = 4

In [ ]:
# implement game
def create_board():
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))  # matrix with zero elements
    return board

In [ ]:
# implement game
def drop_piece(board , row , col , piece):
    board[row][col] = piece  
    # بيملي المكان بتاع العمود و الصف ده بالقيمة اللي اللاعب دخلها

In [ ]:
# implement game
def is_valid_location(board , col):       
    return board[ROW_COUNT-1][col] == 0  #ROW_COUNT = 6 - 1 = 5 
    # بشوف الصف اللي فوق خالص للعمود اللي هو مختاره لو كان العمود بتاع الصف ده مش بيساوي 0 يبقي
    #                                 كده العمود ده اتملي و مينفعش يضيف فيه القيمه اللي هو عايزها 

In [ ]:
# implement game
def get_next_open_row(board , col):
    for r in range(ROW_COUNT): #ROW_COUNT = 6
        if board[r][col] == 0 :
            return r
    # بيشوف الصف بتاع العمود اللي عايزه فاضي ولا لاء لو فاضي يرجعلي هو رقم الصف ده

In [ ]:
# implement game
def print_board(board):
    print(np.flip(board , 0))  # flip() take two arguments array , axis/optional(None , int , tuple)
    # axis = 0 : هتعكسها كانها واقفة قدام مرايه
    # عشان يظهرلي أنها بتتملي من تحت لفوق 

In [ ]:
# implement game
def winning_move(board , piece):
    # check horizontal locations for win
    for c in range(COLUMN_COUNT-3): #COLUMN_COUNT = 7 - 3 = 4  range : 0 -> 3  خليتها 4 عشان هو هيكسب بكده
        for r in range(ROW_COUNT):  #ROW_COUNT = 6    range : 0 -> 5   هيمشي علي كل صف لحد ما يلاقي المكسب بتاعه
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece :
                                                                               # بخليه يشوف ال 4 عواميد اللي ورا بعض
                return True  # يعني كسب 
   
    # check vertical locations for win 
    for c in range(COLUMN_COUNT):#COLUMN_COUNT = 7          range : 0 -> 6
        for r in range(ROW_COUNT-3):#ROW_COUNT = 6 - 3 = 3   range : 0 -> 2
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece :
                return True

    # check positively sloped diagonals
    for c in range(COLUMN_COUNT-3): #COLUMN_COUNT = 7 - 3 = 4  range : 0 -> 3 
        for r in range(ROW_COUNT-3): #ROW_COUNT = 6 - 3 = 3   range : 0 -> 2
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece :
                return True

    # check negatively sloped diagonals
    for c in range(COLUMN_COUNT-3): #COLUMN_COUNT = 7 - 3 = 4  range : 0 -> 3
        for r in range(3 , ROW_COUNT): #ROW_COUNT = 6   range : 3 -> 5
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece :
                return True

In [ ]:
#AI
def evaluate_window(window , piece): # score بيحكسب ال 
    score = 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE :
        opp_piece = AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

In [ ]:
#AI
def score_position(board , piece):
    score = 0

    ## score center column 
    center_array = [int(i) for i in list(board[: , COLUMN_COUNT//2])] # :  every row positin
    center_count = center_array.count(piece)
    score += center_count*3

    ## score Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT - 3): # i want to see 4 columns only in each time
            window = row_array[c:c+WINDOW_LENGTH] #WINDOW_LENGTH = 4
            score += evaluate_window(window , piece)

    ## score vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r+WINDOW_LENGTH] 
            score += evaluate_window(window , piece)

    ## score  positive sloped diagonal
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            # board[r+i][c+i] for i in range(WINDOW_LENGTH) هيزيد الصف واحد و العمود واحد كل مرة 
            # بالظبط winning_move method زي  
            score += evaluate_window(window , piece)

    ## score negative sloped diagonal
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3): # from top to down
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            # board[r+3-i][c+i] for i in range(WINDOW_LENGTH) هيبدا من بعد الصف ال 3 هينقص الصف واحدو العمود واحد كل مره
            # بالظبط winning_move method زي 
            score += evaluate_window(window , piece)

    return score

In [ ]:
#AI
def is_terminal_node(board): # there is player win or the game is over
    return winning_move(board , PLAYER_PIECE) or winning_move(board , AI_PIECE) or len(get_valid_locations(board)) == 0

In [ ]:
#AI 
def minmax(board , depth , alpha , beta , maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board , AI_PIECE):
                 return (None , 1000000000000000)
            elif winning_move(board , PLAYER_PIECE):
                 return (None , -1000000000000000)
            else:  # game is over , no more valid moves  , len(get_valid_locations(board)) == 0
                 return (None , 0)
        else: # depth is zero
            return (None , score_position(board , AI_PIECE))

    if maximizingPlayer:
         value = -math.inf  # alpha
         column = random.choice(valid_locations)
         for col in valid_locations:
            row = get_next_open_row(board , col)
            b_copy = board.copy() 
            drop_piece(b_copy , row , col , AI_PIECE)
            new_score = minmax(b_copy , depth-1 , alpha , beta , False)[1] # first index
            if new_score > value :
                value = new_score
                column = col
            alpha = max(alpha , value)
            if alpha >= beta:
                break
         return column , value

    else: # minimizing player
        value = math.inf # beta
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board , col)
            b_copy = board.copy()
            drop_piece(b_copy , row , col , PLAYER_PIECE)   
            new_score = minmax(b_copy , depth-1 , alpha , beta , True)[1] # first index
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta , value)
            if alpha >= beta :
                break
        return column , value  

In [ ]:
#AI
def get_valid_locations(board):
    var valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board , col):
            valid_locations.append(col)
    return valid_locations #فيها الاماكن الفاضية عندي array هيرجعلي 

In [ ]:
#AI
def pick_best_move(board , piece): # بيدور علي افضل مكان ان هو يلعب فيه
    valid_locations = get_valid_locations(board)
    best_score = -10000
    best_col = random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board , col)
        temp_board = board.copy() # memory location بيجرب فيها
        drop_piece(temp_board , row , col , piece)
        score = score_position(temp_board , piece)
        if score > best_score:
            best_score = score
            best_col = col
            
    return best_col 

In [ ]:
# implement shape or graphics of the game

def draw_board(board): # like print board but with pygame graphics
    for c in range(COLUMN_COUNT): #COLUMN_COUNT = 7
        for r in range(ROW_COUNT): #ROW_COUNT = 6
           # create a blue rectangle to be the base of the game length = 700 and width = 100
           #                                       700           600+100 = 700             100         100
            pygame.draw.rect(screen , BLUE , (c*SQUARESIZE , r*SQUARESIZE+SQUARESIZE , SQUARESIZE , SQUARESIZE)) # take four sizes 
            # create a small black circles to make the player see how many circles is empty(surface , color , position , radius)
            #                                                  position = ((c*100) + 50 , (r*100) + 100 + 50)                    45
            pygame.draw.circle(screen , BLACK , (int(c*SQUARESIZE+SQUARESIZE/2) , int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)) , RADIUS) 
            #                                                فبيطلع معايا ايرور float عشان هي بتطلعلي int حطيت   

    #height = (ROW_COUNT + 1)* SQUARESIZE # 700 px ضيفت صف عشان خاطر مكان الكورة اللي بتتحرك فوق 
    # matrix لل flip عشان انا عامل height - row  عملت 
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == PLAYER_PIECE: #PLAYER_PIECE = 1
                # create a small red circle for player 1 to know him that he plays her . (surface , color , position , radius)
                pygame.draw.circle(screen , RED , (int(c*SQUARESIZE+SQUARESIZE/2) , height - int(r*SQUARESIZE+SQUARESIZE/2)) , RADIUS)
            elif board[r][c] == AI_PIECE: #AI_PIECE = 2
                # create a small yellow circle for player 2 to know him that he plays her . (surface , color , position , radius)
                pygame.draw.circle(screen , YELLOW , (int(c*SQUARESIZE+SQUARESIZE/2) , height - int(r*SQUARESIZE+SQUARESIZE/2)) , RADIUS)
        pygame.display.update()

In [ ]:
board = create_board()  # create matrix(6x7) with zero elemnets
print_board(board)
game_over = False  # no player win the game

pygame.init() # initialize all imported pygame modules is a convenient way to get everything started .

SQUARESIZE = 100
width = COLUMN_COUNT * SQUARESIZE # 700 px
height = (ROW_COUNT + 1)* SQUARESIZE # 700 px ضيفت صف عشان خاطر مكان الكورة اللي بتتحرك فوق 

size = (width , height) # size of the screen

RADIUS = int(SQUARESIZE/2 - 5) # radius of the circle 100/2 = 50 -5 = 45

screen = pygame.display.set_mode(size) # بظهر المربع بتاعي الكبير كده
draw_board(board)
pygame.display.update()

myfont = pygame.font.SysFont("monospace" , 75) # نوع و حجم الخط اللي هكتب بيه

turn = random.randint(PLAYER , AI)   # make the start random if 0 ask for a player 1 if 1 ask for a player 2

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
while not game_over:  # stop when one player is win

    for event in pygame.event.get(): # عندي mouse هنا انا باخد تحركات ال 
        if event.type == pygame.QUIT: # يعني عايز يخرج من اللعبه خالص
            sys.exit()

        if event.type == pygame.MOUSEMOTION: # عشان اشوف تحركات الكورة و انزلها في المكان اللي انا عايزاه
            pygame.draw.rect(screen , BLACK , (0 , 0 , width , SQUARESIZE))
            posx = event.pos[0]
            if turn == PLAYER: # player = 0
                pygame.draw.circle(screen , RED , (posx , int(SQUARESIZE/2)) , RADIUS)
            pygame.display.update()

        if event.type == pygame.MOUSEBUTTONDOWN: # care about position and button
            pygame.draw.rect(screen , BLACK , (0 , 0 , width , SQUARESIZE))

            #Ask for player 1 input
            if turn == PLAYER: # player = 0
                posx = event.pos[0]
                col = int(math.floor(posx/SQUARESIZE)) # between 0 and 100

                if is_valid_location(board , col): # بيشوف العمود اللي دخله ده الصف بتاعه فاضي ولا لاء
                     row = get_next_open_row(board , col) # بيروح يشوف اخر صف عندي كام مملي عند العمود ده مين 
                                                          #                    و يبعتلي اللي بعده عشان اضيف فيه
                     drop_piece(board , row , col , PLAYER_PIECE) # PLAYER_PIECE = 1 .يحط القيمه بتاعته في الصف اللي هو جايبه من فوق 

                     if winning_move(board , PLAYER_PIECE):# PLAYER_PIECE = 1
                         label = myfont.render("player 1 wins!!" , 1 , RED)
                         screen.blit(label , (40 , 10))
                         game_over = True # player 1 win the game

                     turn += 1 # add 1 to the turn then get modulus of it 
                     turn = turn % 2 #between(0 and 1)  if mod = 0 then it ask for player 1 else mod = 1 then it ask for player 2

                     print_board(board)
                     draw_board(board)

    # Ask for player 2 input
    if turn == AI and not game_over: # AI = 1 and game_over not equal true
        col , minimax_score = minmax(board , 6 , -math.inf , math.inf , True)

        if is_valid_location(board , col): # بيشوف العمود اللي دخله ده الصف بتاعه فاضي ولا لاء
             row = get_next_open_row(board , col)# بيروح يشوف اخر صف عندي كام مملي عند العمود ده مين 
                                                 #                             و يبعتلي اللي بعده عشان 
             drop_piece(board , row , col , AI_PIECE) # AI_PIECE = 2 .يحط القيمه بتاعته في الصف اللي هو جايبه من فوق 


             if winning_move(board , AI_PIECE):# AI_PIECE = 2
                 label = myfont.render("player 2 wins!!" , 1 , YELLOW)
                 screen.blit(label , (40 , 10))
                 game_over = True # player 2 win the game

             print_board(board)
             draw_board(board)

             turn += 1 # add 1 to the turn then get modulus of it 
             turn = turn % 2 #between(0 and 1)  if mod = 0 then it ask for player 1 else mod = 1 then it ask for player 2

    if game_over:
         pygame.time.wait(3000) #milisecond    -> 3 seconds


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 2. 1. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]
 [0. 0. 2. 1. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]
 [0. 0. 2. 1. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]
 [0. 0. 2.